In [ ]:
#!pip install yfinance

In [ ]:
# Importing Necesssary Libraries
import pandas as pd
import numpy as np
import yfinance as yf
from google.colab import drive
from google.colab import files
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the file path in your Google Drive
file_path = '/content/drive/MyDrive/NLP/HS/New_Data/abcnews-date-text.csv'

# Read the file into a DataFrame
df = pd.read_csv(file_path)

In [ ]:
df.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [ ]:
# Change it to a datetime format
df['publish_date'] = pd.to_datetime(df['publish_date'], format='%Y%m%d')

In [ ]:
# Define the ticker symbols for S&P 500
sp500_ticker = "^GSPC"

# Download historical data for S&P 500 from Yahoo Finance
sp500_data = yf.download(sp500_ticker, start=df['publish_date'].min(), end=df['publish_date'].max())

# Reset the index to make 'Date' a column for merging
sp500_data.reset_index(inplace=True)

# Merge the two DataFrames based on the 'publish_date' column
merged_df = pd.merge(df, sp500_data[['Date', 'Open', 'High', 'Low', 'Close']], left_on='publish_date', right_on='Date', how='left')

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Display the updated DataFrame
merged_df.head()

,publish_date,headline_text,Date,Open,High,Low,Close
0,2003-02-19,aba decides against community broadcasting lic...,2003-02-19,851.169983,851.169983,838.789978,845.130005
1,2003-02-19,act fire witnesses must be aware of defamation,2003-02-19,851.169983,851.169983,838.789978,845.130005
2,2003-02-19,a g calls for infrastructure protection summit,2003-02-19,851.169983,851.169983,838.789978,845.130005
3,2003-02-19,air nz staff in aust strike for pay rise,2003-02-19,851.169983,851.169983,838.789978,845.130005
4,2003-02-19,air nz strike to affect australian travellers,2003-02-19,851.169983,851.169983,838.789978,845.130005


In [ ]:
# Create DataFrame for S&P
df = merged_df[['publish_date', 'headline_text', 'Open', 'High', 'Low', 'Close']].copy()

In [ ]:
# Display the S&P DataFrame
df.head()

,publish_date,headline_text,Open,High,Low,Close
0,2003-02-19,aba decides against community broadcasting lic...,851.169983,851.169983,838.789978,845.130005
1,2003-02-19,act fire witnesses must be aware of defamation,851.169983,851.169983,838.789978,845.130005
2,2003-02-19,a g calls for infrastructure protection summit,851.169983,851.169983,838.789978,845.130005
3,2003-02-19,air nz staff in aust strike for pay rise,851.169983,851.169983,838.789978,845.130005
4,2003-02-19,air nz strike to affect australian travellers,851.169983,851.169983,838.789978,845.130005


In [ ]:
# Add a column for intraday movement in df
# Upward movement >> 1 & Downward movement
df['intraday_movement'] = (df['Close'] > df['Open']).astype(int)
df

,publish_date,headline_text,Open,High,Low,Close,intraday_movement
0,2003-02-19,aba decides against community broadcasting lic...,851.169983,851.169983,838.789978,845.130005,0
1,2003-02-19,act fire witnesses must be aware of defamation,851.169983,851.169983,838.789978,845.130005,0
2,2003-02-19,a g calls for infrastructure protection summit,851.169983,851.169983,838.789978,845.130005,0
3,2003-02-19,air nz staff in aust strike for pay rise,851.169983,851.169983,838.789978,845.130005,0
4,2003-02-19,air nz strike to affect australian travellers,851.169983,851.169983,838.789978,845.130005,0
...,...,...,...,...,...,...,...
1244179,2021-12-31,two aged care residents die as state records 2...,NaN,NaN,NaN,NaN,0
1244180,2021-12-31,victoria records 5;919 new cases and seven deaths,NaN,NaN,NaN,NaN,0
1244181,2021-12-31,wa delays adopting new close contact definition,NaN,NaN,NaN,NaN,0
1244182,2021-12-31,western ringtail possums found badly dehydrate...,NaN,NaN,NaN,NaN,0


In [ ]:
# Drop rows where Open or Close is NaN
df = df.dropna(subset=['Open', 'Close'])

# Get the number of rows and columns
num_rows, num_columns = df.shape

# Display the result
print("Number of Rows:", num_rows)
print("Number of Columns:", num_columns)

Number of Rows: 1020055
Number of Columns: 7


In [ ]:
# Count occurrences of each unique value in 'intraday_movement' column
counts = df['intraday_movement'].value_counts()

# Display the counts
print("Number of 1s (Upward Movement):", counts[1])
print("Number of 0s (No Movement or Downward Movement):", counts[0])

Number of 1s (Upward Movement): 557808
Number of 0s (No Movement or Downward Movement): 462247


In [ ]:
# Create a new DataFrame with only 'publish_date', 'headline_text', and 'intraday_movement'
new_df = df[['publish_date', 'headline_text', 'intraday_movement']].copy()

new_df.head()

,publish_date,headline_text,intraday_movement
0,2003-02-19,aba decides against community broadcasting lic...,0
1,2003-02-19,act fire witnesses must be aware of defamation,0
2,2003-02-19,a g calls for infrastructure protection summit,0
3,2003-02-19,air nz staff in aust strike for pay rise,0
4,2003-02-19,air nz strike to affect australian travellers,0


In [ ]:
# Convert 'publish_date' to datetime format if it's not already
df['publish_date'] = pd.to_datetime(df['publish_date'])

# Subset the dataframe to include only values after 2019
df = df[df['publish_date'] > '2019-01-01']

# Display the subsetted dataframe
df

<ipython-input-22-994dae19be10>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['publish_date'] = pd.to_datetime(df['publish_date'])


,publish_date,headline_text,Open,High,Low,Close,intraday_movement
1152019,2019-01-02,adelaide man arrested over police officer hit ...,2476.959961,2519.489990,2467.469971,2510.030029,1
1152020,2019-01-02,aged care watchdog formed promises improvement...,2476.959961,2519.489990,2467.469971,2510.030029,1
1152021,2019-01-02,american detained in russia on spying charge i...,2476.959961,2519.489990,2467.469971,2510.030029,1
1152022,2019-01-02,australia must be ready for cave rescues exper...,2476.959961,2519.489990,2467.469971,2510.030029,1
1152023,2019-01-02,australian housing prices fall 4.8pc weakest s...,2476.959961,2519.489990,2467.469971,2510.030029,1
...,...,...,...,...,...,...,...
1244142,2021-12-30,sydney testing clinics stretches back kilometres,4794.229980,4808.930176,4775.330078,4778.729980,0
1244143,2021-12-30,those sitting on the fence are now rolling up,4794.229980,4808.930176,4775.330078,4778.729980,0
1244144,2021-12-30,truly; utterly heartbreaking inside afghanistan,4794.229980,4808.930176,4775.330078,4778.729980,0
1244145,2021-12-30,waiting times for covid testing clinics balloo...,4794.229980,4808.930176,4775.330078,4778.729980,0


In [ ]:
df.to_csv('/content/drive/MyDrive/NLP/HS/New_Data/s&p_1.csv', index=False)